## Connect Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


## Imports

In [2]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Data

In [3]:
# Directory with our training pictures
train_dir = '/content/drive/MyDrive/Major Project/Currency Notes Recognition/final_dataset/training'

# Directory with our validation pictures
validation_dir = '/content/drive/MyDrive/Major Project/Currency Notes Recognition/final_dataset/validation'

## Importing Model

In [4]:
# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [5]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')

87924736/87910968 [==============================] - 1s 0us/step


In [6]:
for layer in pre_trained_model.layers:
  layer.trainable = False

## Defining Callback

In [7]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.959):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [8]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(8, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [9]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 45,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                  #  shear_range = 0.2,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'categorical', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'categorical', 
                                                          target_size = (150, 150))

Found 7833 images belonging to 8 classes.
Found 536 images belonging to 8 classes.


In [12]:
# callbacks = myCallback()
filepath = '/content/drive/MyDrive/Major Project/Currency Notes Recognition/Trained Model/model_3.h5'

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 390, #7833/20
            epochs = 20,
            validation_steps = 27, #536/20
            verbose = 1,
            callbacks=[checkpoint])

Epoch 1/20
  4/390 [..............................] - ETA: 5:28 - loss: 0.7142 - acc: 0.8250

KeyboardInterrupt: ignored

In [11]:
model.save_weights('/content/drive/MyDrive/Major Project/Currency Notes Recognition/Trained Model/weights_3.h5')